In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [4]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating 01b7e22..a4e19c9
Fast-forward
 useEUCL.ipynb        | 252 ++++++++++++++++++++++++++-------------
 useTripleLoss .ipynb | 328 ++++++++++++++++++++++++++++++++-------------------
 2 files changed, 375 insertions(+), 205 deletions(-)


In [5]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install -q tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [7]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [8]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [68]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [14]:
def get_model():
    shared_embedding_layer1 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="first")
    shared_embedding_layer2 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="second")

    left_input = tf.keras.Input(shape=(), dtype=tf.string)
    right_input = tf.keras.Input(shape=(), dtype=tf.string)

    embedding_left_output= shared_embedding_layer1(left_input)
    a = tfl.Dropout(0.3)(embedding_left_output)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dropout(rate=0.2)(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    # a = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)

    embedding_right_output= shared_embedding_layer2(right_input)
    b = tfl.Dropout(0.3)(embedding_right_output)
    b = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(b)
    b = tfl.BatchNormalization()(b)
    b = tfl.Activation('relu')(b)
    # b = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(b)

    distance = tfl.Lambda(euclidean_distance)([a, b])
    model = Model([left_input, right_input], distance)
    return model

In [15]:
model = get_model()

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 first (KerasLayer)             (None, 512)          256797824   ['input_3[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 512)          0           ['first[0][0]']                  
                                                                                                  
 dense_3 (Dense)                (None, 64)           32832       ['dropout_3[0][0]']              
                                                                                            

In [17]:
model.compile(optimizer="adam", loss = contrastive_loss, metrics = [tf.metrics.BinaryAccuracy()])

In [41]:
loss_history = model.fit(siam_gen([df["texts"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["text"], df_test["reason"]], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/30
50/50 [==============================] - 19s 371ms/step - loss: 0.2992 - binary_accuracy: 0.4069 - val_loss: 0.2953 - val_binary_accuracy: 0.3734
Epoch 2/30
50/50 [==============================] - 17s 352ms/step - loss: 0.2924 - binary_accuracy: 0.4016 - val_loss: 0.2947 - val_binary_accuracy: 0.3891
Epoch 3/30
50/50 [==============================] - 14s 289ms/step - loss: 0.2885 - binary_accuracy: 0.3941 - val_loss: 0.2905 - val_binary_accuracy: 0.3898
Epoch 4/30
50/50 [==============================] - 15s 296ms/step - loss: 0.2816 - binary_accuracy: 0.3883 - val_loss: 0.2911 - val_binary_accuracy: 0.4016
Epoch 5/30
50/50 [==============================] - 18s 371ms/step - loss: 0.2797 - binary_accuracy: 0.3833 - val_loss: 0.2766 - val_binary_accuracy: 0.3711
Epoch 6/30
50/50 [==============================] - 14s 288ms/step - loss: 0.2723 - binary_accuracy: 0.3731 - val_loss: 0.2727 - val_binary_accuracy: 0.3789
Epoch 7/30
50/50 [==============================] - 18s 36

In [73]:
y_preds = model.predict([df["texts"], df["reason"]])
y_preds = tf.math.sigmoid(1 - y_preds).numpy()
accuracy(df["label"],y_preds)

0.9982924746920356

In [74]:
max(y_preds), min(y_preds)

(array([0.73099643], dtype=float32), array([0.44637114], dtype=float32))

In [75]:
from sklearn.metrics import brier_score_loss as brier_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

In [76]:
y_hat = (y_preds >= 0.5).astype(int)

In [77]:
print("Brier Loss                       : ", brier_loss(df['label'], 1 - y_preds))
print("Precision Score                  : ", precision_score(df['label'], y_hat))
print("Recall Score                     : ", recall_score(df['label'], y_hat))
print("F1 Score                         : ", f1_score(df['label'], y_hat))
# print("Roc AUC Score                    : ", roc_auc_score(df['label'], y_preds))
# print("BCE Loss                         : ", log_loss(df['label'], y_preds))

Brier Loss                       :  0.46022743849172487
Precision Score                  :  1.0
Recall Score                     :  0.9982924746920356
F1 Score                         :  0.9991455078125


In [69]:
y_preds = model.predict([df_test["text"], df_test["reason"]])
y_preds = tf.math.sigmoid(1 - y_preds).numpy()
accuracy(df_test["label"], y_preds)

0.36477777777777776

In [70]:
max(y_preds), min(y_preds)

(array([0.73099643], dtype=float32), array([0.4189163], dtype=float32))

In [71]:
y_hat = (y_preds >= 0.5).astype(int)

In [72]:
print("Brier Loss                       : ", brier_loss(df_test['label'], y_preds))
print("Precision Score                  : ", precision_score(df_test['label'], y_hat))
print("Recall Score                     : ", recall_score(df_test['label'], y_hat))
print("F1 Score                         : ", f1_score(df_test['label'], y_hat))
# print("Roc AUC Score                    : ", roc_auc_score(df_test['label'], y_preds))
print("BCE Loss                         : ", log_loss(df_test['label'], 1 - y_preds))

Brier Loss                       :  0.3028793215974457
Precision Score                  :  0.33997171812396887
Recall Score                     :  0.9613462179273575
F1 Score                         :  0.5023069556890398
BCE Loss                         :  0.6678047352068954


In [ ]:
model.save("models/useEUCL/")